# Use Case Description

Using public job postings of engineers as OSINT is a common practice. <br>
In this use case, we will analyze a hypothetical DevOps engineer job description to uncover potential exploitation scenarios.

## Model used for this use case
Reasoning Model is well-suited for this use case because the task is complex and requires deep reasoning. <br>
The Reasoning Model is currently in preview. If you don't have access, you can use the instruct model instead.

## SetUp

The setup scripts below are essentially the same as those in the [Quickstart (Reasoning Model)](https://github.com/RobustIntelligence/foundation-ai-cookbook/blob/main/1_quickstarts/Preview_Quickstart_reasoning_model.ipynb)

### Notice
- The code below assumes that users have access to the models via Hugging Face. If you are using API access instead, please replace the inference code with the API version provided in the Quickstart guide.
- This model is currently in preview mode and may receive updates. As a result, outputs can vary even when parameters are configured to ensure reproducibility.

In [1]:
import os
import transformers
import torch
from IPython.display import display, Markdown

HF_TOKEN = os.getenv("HF_TOKEN")

def _get_device():
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

DEVICE = _get_device()

In [2]:
MODEL_ID = "" # To be relaced with the final model name

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float32, # this model's tensor_type is float32
    token=HF_TOKEN,
)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
generation_args = {
    "max_new_tokens": 2048,
    "temperature": None,
    "repetition_penalty": 1.2,
    "do_sample": False,
    "use_cache": True,
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.pad_token_id,
}

In [4]:
import re

def inference(prompt):
    messages = [
        {"role": "user", "content": prompt},
    ]
    inputs = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # The old model version didn’t include the <think> token in the chat template.
    think_token = "<think>\n"
    if not inputs.endswith(think_token):
        inputs += think_token
    
    inputs = tokenizer(inputs, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **generation_args,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens = False)

    # extract the thinking part only
    match = re.search(r"<think>(.*?)<\|end_of_text\|>", response, re.DOTALL)
    
    return match.group(1).strip()

## Generate a mock job description as OSINT to be analyzed

Let’s first create a mock job description, which can also be generated using our reasoning model. <br>
If you have a specific job description in mind or want to use a real one found online, replace the ```job_description``` variable with your data.

In [5]:
prompt = '''You are a recruiter at Cisco, hiring a Senior DevOps Engineer for the Security Products team in San Jose. 
Write a comprehensive and detailed job description for this role in Markdown format.'''

job_description = inference(prompt)
display(Markdown(job_description))

# Senior DevOps Engineer - Security Products Team

## Job Overview

As a Senior DevOps Engineer on the Security Products team at Cisco in San Jose, you will play a critical role in driving the development and deployment of cutting-edge security solutions. You'll collaborate closely with cross-functional teams to automate processes, improve infrastructure efficiency, and ensure high-quality delivery of our products. This position requires strong leadership skills, deep technical expertise, and a passion for innovation.

## Key Responsibilities

- **Design and Implement Infrastructure Solutions:** Develop scalable and secure infrastructure architectures using tools like Kubernetes, Docker, Terraform, and Ansible. Optimize CI/CD pipelines to enhance automation and reduce deployment times.
  
- **Collaborate Across Teams:** Work seamlessly with developers, security engineers, product managers, and other stakeholders to define and implement best practices that support agile software development life cycles.
  
- **Monitor and Improve Performance:** Utilize monitoring tools (e.g., Prometheus, Grafana) to track system performance and proactively identify areas for improvement. Drive initiatives to increase operational efficiency and reliability.
  
- **Security Hardening & Compliance:** Ensure all systems adhere to industry standards and compliance requirements such as NIST, SOC, etc. Contribute to creating and maintaining security policies within DevOps frameworks.
  
- **Incident Management & Troubleshooting:** Lead incident response efforts when necessary, working with teams to diagnose issues, apply fixes, and document lessons learned. Continuously refine troubleshooting methodologies.
  
- **Automation Strategy Development:** Define and evolve the automation strategy for the team, identifying opportunities to leverage AI/ML where applicable to further streamline operations.
  
- **Mentorship & Leadership:** Mentor junior DevOps Engineers, sharing your knowledge and experience. Lead by example, fostering a culture of collaboration, continuous learning, and excellence.
  
- **Stay Updated on Industry Trends:** Keep abreast of emerging technologies, trends, and best practices in DevOps and cloud-native applications. Influence the adoption of new tools and methods as appropriate.

## Qualifications

**Required:**

- Bachelor’s degree in Computer Science, Information Technology, or related field; or equivalent practical experience.
- 7+ years of professional experience in DevOps, Systems Engineering, or relevant field.
- Strong understanding of containerization (Docker), orchestration (Kubernetes), and cloud platforms (AWS/GCP/Azure).
- Proficiency in scripting languages (Python, Bash) and configuration management tools (Ansible, Puppet, Chef).
- Experience with version control systems (Git), CI/CD tools (Jenkins, CircleCI, GitLab CI), and automated testing frameworks.
- Deep knowledge of network security principles and their application in infrastructure design.
- Excellent problem-solving abilities and attention to detail.
- Ability to work effectively in a fast-paced, dynamic environment.
- Effective communication and interpersonal skills; capable of leading meetings and presenting ideas to diverse audiences.

**Preferred:**

- Certification in DevOps, Cloud, or Cybersecurity (e.g., AWS Certified DevOps Professional, CCNA, CNCF certifications)
- Experience with SRE principles and tooling (like Stackdriver, ELK stack)
- Familiarity with Agile and Scrum methodologies
- Previous mentorship or leadership roles within engineering teams
- Knowledge of SIEM systems, IDS/IPS, and endpoint protection

## Physical Requirements

This position is based in San Jose, CA. The selected candidate must be willing to work onsite according to Cisco's needs.

## Application Process

To apply, please submit your resume along with a cover letter detailing why you're excited about this opportunity and how your experiences align with the responsibilities outlined above. Include specific examples of projects you've led or contributed to that demonstrate your qualifications.

---

*Note:* Cisco is an equal-opportunity employer committed to diversity and inclusion. We welcome applicants from all backgrounds and do not discriminate based on race, color, religion, sex (including pregnancy, transgender status, gender identity, sexual orientation), national origin, family medical history, genetic information, disability, age, veteran status, political affiliation, or any other basis protected under law.

### Analyze the job description to find potential exploitation scenarios

Let’s have the model analyze the job description and generate a report. <br>
It’s recommended to provide the model with a clear format to follow for consistency and clarity.

In [6]:
prompt = f'''You are a cybersecurity expert outside Cisco and wants to run some tests against their products only using public information.
Based on your knowledge and experience of OSINT, analyze the job description below and write a report of potential exploitation for penetration tests in the following formats.

##Summary
##Technical Findings
##Potential exploitations


<Job Description>
{job_description}
'''

In [7]:
response = inference(prompt)
display(Markdown(response))

Report on Potential Exploitation Opportunities Based on Job Description – Senior DevOps Engineer, Security Products Team, Cisco

### Summary
The provided job description outlines key responsibilities and qualifications required for a Senior DevOps Engineer on Cisco's Security Products team. While the primary focus is on designing and implementing secure infrastructures, automating processes, and ensuring compliance with standards like NIST/SOC, there exist potential vulnerabilities and attack vectors that could be exploited during penetration tests. These findings aim to inform red-team exercises, emphasizing areas where attackers might target given the described role and technology stack.

### Technical Findings

1. **Containerization and Orchestration (Docker/Kubernetes):**
   - **Vulnerability**: Misconfigured containers or orchestrators can lead to privilege escalation. For instance, default configurations may allow unnecessary privileges, or images pulled from untrusted sources might contain backdoors.
   - **Mitigation**: Ensuring proper hardening, least privilege access, and scanning dependencies regularly.

2. **Cloud Platforms (AWS/GCP/Azure):**
   - **Vulnerability**: Inadequate IAM policies or misconfigured storage buckets can expose sensitive data. Also, unused services or resources might remain active, increasing attack surfaces.
   - **Mitigation**: Regular audits, principle of least privilege, and removing stale resources.

3. **CI/CD Pipelines (Jenkins/CircleCI/GitLab CI):**
   - **Vulnerability**: Weaknesses in pipeline scripts, insecure artifact repositories, or lack of secrets management can lead to code injection or unauthorized deployments.
   - **Mitigation**: Secure coding practices, secret vault integration, and static analysis tools.

4. **Monitoring Tools (Prometheus/Grafana):**
   - **Vulnerability**: If these tools aren't properly secured, they could become entry points for attacks. E.g., exploiting dashboards to inject malicious queries or accessing underlying databases without authentication.
   - **Mitigation**: Proper access controls, encryption of data at rest and transit, and regular updates.

5. **Compliance and Policy Enforcement:**
   - **Vulnerability**: Gaps in adherence to NIST/SOC guidelines, especially if automated checks aren’t robust enough. Manual processes here could introduce human error.
   - **Mitigation**: Comprehensive policy-as-code approaches (IaC + CIS benchmarks) integrated into CI/CD, coupled with continuous auditing.

6. **Automated Incident Response and ML-driven Automation:**
   - **Vulnerability**: Over-reliance on AI/ML models without considering adversarial techniques could lead to false negatives. Additionally, poorly designed IR playbooks might leave room for evasion.
   - **Mitigation**: Testing AI components with known adversarial inputs, ensuring diversified detection mechanisms, and having manual review checkpoints.

### Potential Exploitations

1. **Supply Chain Attack via Compromised Docker Images or Dependencies:**
   - Targeting publicly available Docker hubs, replacing legitimate images with tampered ones containing malware. Given the mention of pulling images from external registries, verifying integrity through digital signatures or Notary would be crucial.

2. **Privilege Escalation in Kubernetes Clusters:**
   - Abusing RBAC permissions, pod security contexts, or node-level access. Since the engineer works with Kubernetes, ensuring strict Role-Based Access Control (RBAC) and Pod Security Policies (PSPs) is essential.

3. **Misconfiguration Attacks in IaaS Services:**
   - Identifying exposed storage buckets, overly permissive IAM roles, or leftover test environments. Penetration testers should look for signs of inadequate compartmentalization and segmentation.

4. **Code Injection in CI/CD Pipelines:**
   - Manipulating build artifacts or injecting malicious steps into Jenkinsfiles or similar config files. Inspecting the source code management and build process for weaknesses in validation and sanitization.

5. **Data Exfiltration via Monitoring Tool Dashboards:**
   - Crafting SQLi or XSS payloads in Grafana panels to extract database contents. Alternatively, abusing API keys stored in configs or environment variables accessible through logs.

6. **Bypassing Security Controls via Adversarial Machine Learning:**
   - Developing scenarios where ML-based anomaly detectors are tricked into ignoring actual threats. Testing the model's resilience against poisoning or evasion techniques.

In conducting penetration tests aligned with this job description, it's vital to stay updated on the latest CVEs affecting mentioned technologies and simulate real-world attacks. Collaboration with blue teams to address found vulnerabilities ensures that Cisco's Security Products team enhances its defenses accordingly.